In [ ]:
"""
Created on Wed Apr 15 09:53 2022

Apply model coming from train_DNN_with_prepared_input.ipynb on one other run

Author: @claraburgard

"""

In [ ]:
import numpy as np
import xarray as xr
from tqdm.notebook import trange, tqdm
import glob
import matplotlib as mpl
import seaborn as sns
import datetime
import time
import os,sys

import tensorflow as tf
from tensorflow import keras
from contextlib import redirect_stdout

from basal_melt_neural_networks.constants import *
import basal_melt_neural_networks.diagnostic_functions as diag
import basal_melt_neural_networks.data_formatting as dfmt

READ IN DATA

In [ ]:
inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'
outputpath_nn_models = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/'
outputpath_doc = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/custom_doc/'

In [ ]:
nemo_run0 = 'OPM006' #['OPM006', 'OPM016', 'OPM018', 'OPM021', 'OPM026', 'OPM027', 'OPM031-1', OPM031-2']
if nemo_run0 in ['OPM031-1','OPM031-2']:
    nemo_run = 'OPM031'
else:
    nemo_run = nemo_run0
    
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'

In [ ]:
file_isf = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')

In [ ]:
isf_list = [10, 11, 12, 13, 18, 21, 22, 23, 24, 25, 30, 31, 33, 38, 39, 
            40, 42, 43, 44, 45, 47, 48, 51, 52, 53, 54, 55, 58, 61, 65, 
            66, 69, 70, 71, 73, 75]

In [ ]:
for timetag in ['20220427-1059','20220427-1021','20220427-1042','20220427-1051']:
    
    
    new_path_output = outputpath_melt_nn+timetag+'/'
    if not os.path.isdir(new_path_output):
        print("I did not find this folder "+timetag+") so I created a new one, I hope that's ok!")
        os.mkdir(new_path_output)
    else:
        print("This folder ("+timetag+") exists already!")

    new_path_model = outputpath_nn_models+timetag+'/'
    if not os.path.isdir(new_path_model):
        print("I did not find this folder ("+timetag+") in model folder so I created a new one, I hope that's ok!")
    
    print(timetag)
    if timetag in ['20220427-0957','20220427-1052','20220427-1058','20220427-1059']:
        timetag_data = '20220427-0957'
        path_data = inputpath_data+'EXTRAPOLATED_ISFDRAFT/'

    elif timetag in ['20220427-1002','20220427-1021','20220427-1042','20220427-1051']:
        timetag_data = '20220427-1002'
        path_data = inputpath_data+'WHOLE_PROF/'
        

    norm_data_path = outputpath_nn_models+timetag_data+'/'

    normalisation_coeff = xr.open_dataset(norm_data_path+ 'dataset_norm_training_factors_'+timetag_data+'.nc')
    model = keras.models.load_model(new_path_model + 'model_nn_'+timetag+'.h5')

    y_all_isf = None

    for kisf in tqdm(isf_list): 

        ### READ DATA
        df_nrun = pd.read_csv(path_data + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+nemo_run0+'.csv',index_col=[0,1,2])
        clean_df_nrun_kisf = pd.read_csv(path_data + 'dataframe_input_isf'+str(kisf).zfill(3)+'_'+nemo_run0+'.csv',index_col=[0,1,2])
        clean_df_nrun_kisf.reset_index(drop=True, inplace=True)
        clean_ds_nrun_kisf = clean_df_nrun_kisf.to_xarray()

        normalisation_coeff_input = normalisation_coeff.drop_vars(['melt_m_ice_per_y'])
        normalised_vars = (clean_ds_nrun_kisf.drop_vars(['melt_m_ice_per_y']) - normalisation_coeff_input.sel(metric='mean_vars'))/normalisation_coeff_input.sel(metric='std_vars')

        input_var = normalised_vars.to_array().load()
        ref_melt = clean_ds_nrun_kisf['melt_m_ice_per_y'].load()

        ### RUN THE MODEL
        y_out_norm = model.predict(input_var.T.values)
        y_out_norm_xr = xr.DataArray(data=y_out_norm.squeeze()).rename({'dim_0': 'index'})
        y_out_norm_xr = y_out_norm_xr.assign_coords({'index': input_var.index})
        y_out = (y_out_norm_xr * normalisation_coeff['melt_m_ice_per_y'].sel(metric='std_vars')) + normalisation_coeff['melt_m_ice_per_y'].sel(metric='mean_vars')

        y_out_pd_s = pd.Series(y_out.values,index=df_nrun.index,name='predicted_melt') 
        y_target_pd_s = pd.Series(ref_melt.values,index=df_nrun.index,name='reference_melt') 

        ### PUT SOME ORDER IN THE FILE
        y_out_xr = y_out_pd_s.to_xarray()
        y_target_xr = y_target_pd_s.to_xarray()
        y_to_compare = xr.merge([y_out_xr.T, y_target_xr.T]).sortby('y')

        y_whole_grid = y_to_compare.reindex_like(file_isf['ISF_mask'])
        if y_all_isf is None:
            y_all_isf = y_whole_grid
        else:
            y_all_isf = y_all_isf.combine_first(y_whole_grid)

    y_all_isf.to_netcdf(new_path_output+'NN_melt_predicted_reference_m_ice_per_yr_'+nemo_run0+'.nc')    

In [ ]:
nemo_run

In [ ]:
plt.scatter(y_all_isf['predicted_melt'],y_all_isf['reference_melt'], alpha=0.01)

In [ ]:
############## OLD STUFF

PREPARE DATA

In [ ]:
normalisation_coeff = xr.open_dataset(new_path_model+ 'dataset_norm_training_factors_'+timetag_data+'.nc')#.set_index('Unnamed: 0')#.drop('Unnamed: 0', 1)
#normalisation_coeff.index.name = None
normalisation_coeff_input = normalisation_coeff.drop_vars(['melt_m_ice_per_y'])

In [ ]:
normalised_vars = (all_input_df.drop_vars(['melt_m_ice_per_y']) - normalisation_coeff_input.sel(metric='mean_vars'))/normalisation_coeff_input.sel(metric='std_vars')

In [ ]:
input_var = normalised_vars.to_array().load()
#ref_melt = all_input_df['melt_m_ice_per_y'].load()

CHOOSE THE RIGHT MODEL

In [ ]:
model = keras.models.load_model(new_path_model + 'model_nn_'+timetag+'.h5')

In [ ]:
y_out_norm = model.predict(input_var.T.values)

In [ ]:
y_out_norm_xr = xr.DataArray(data=y_out_norm.squeeze()).rename({'dim_0': 'index'})
y_out_norm_xr = y_out_norm_xr.assign_coords({'index': input_var.index})

In [ ]:
y_out = (y_out_norm_xr * normalisation_coeff['melt_m_ice_per_y'].sel(metric='std_vars')) + normalisation_coeff['melt_m_ice_per_y'].sel(metric='mean_vars')

In [ ]:
y_out.values.shape

In [ ]:
y_out_pd_s = pd.Series(y_out[:,0],index=df_nrun.index,name='predicted_melt') 
y_target_pd_s = pd.Series(y_val_arr,index=df_nrun.index,name='reference_melt') 

In [ ]:
y_out_xr = y_out_pd_s.to_xarray()
y_target_xr = y_target_pd_s.to_xarray()
y_to_compare = xr.merge([y_out_xr.T, y_target_xr.T]).sortby('y')

In [ ]:
y_to_compare.to_netcdf(new_path_output+'NN_melt_predicted_reference_m_ice_per_yr_'+nemo_run+'.nc')